In [2]:
import requests 
import json
import polyline


In [3]:
def get_routes_json(start,end):
    r = requests.get('https://maps.googleapis.com/maps/api/directions/json?origin='+start+'&destination='+end+'&alternatives=true&key=x')
    j = r.json();
    return j

In [4]:
class Route:
    
    def __init__(self,distance,duration):
        self.distance = distance
        self.duration = duration
        self.steps_list = list()
        self.route_score = 0
        
    def add_Step(self,Step):
        self.steps_list.append(Step)
        
    
class Step:
  
    def __init__(self,distance,lat,long,duration,html,polyline):
        self.distance = distance
        self.duration = duration
        self.html = html
        self.lat = lat
        self.long = long
        self.foodCount = 0
        self.polyline = polyline
        
class Shop:
    
    def __init__(self,lat,lng,shop_id):
        self.lat = lat
        self.lng = lng
        self.shop_id = shop_id
        self.visited = False

def parse_json(json_data):
    count = 0;
    routes_list = [];
    for route in json_data['routes']:
#         num_of_routes = num_of_routes + 1
        for leg in route['legs']:
            route_dist = leg['distance']['value']
            route_dura = leg['duration']['value']
            r = Route(route_dist,route_dura)
            for step in leg['steps']:
                distance = step['distance']['value']
                duration = step['duration']['value']
                lat = step['end_location']['lat']
                long = step['end_location']['lng']
                html = step['html_instructions']
                polyline = step['polyline']['points']
                s = Step(distance,lat,long,duration,html,polyline)
                r.add_Step(s)
        routes_list.append(r)
    
    return routes_list
                

In [5]:
a = get_routes_json('New York','Boston')

In [6]:
routes = parse_json(a)


In [7]:
import math
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1,lon1,lat2,lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance
    print("Result:", distance)






from yelpapi import YelpAPI
from uszipcode import ZipcodeSearchEngine

API_KEY = 'x-x'
Client_ID = 'x'

# set up search for rest based on long and lat
def getCountsForRoute(route):
    total = 0
    searchTerm = "Taco Bell"
    uniques = set()
    final = set()
    shops = list()
    codes = set()
    search_results_list = list()
    yelp_api = YelpAPI(API_KEY)
    search = ZipcodeSearchEngine()
    for step in route.steps_list:
        codes = set()
        if step.distance < 10000:
            res = search.by_coordinate(step.lat, step.long, radius=10, returns=1)
            if len(res) > 0:
                codes.add(str(res[0].Zipcode))
        else:
            miles_stretch = step.distance
            points = polyline.decode(step.polyline)
            new_base = (miles_stretch/len(points))
#             print('this is the new base')
#             print(new_base)
            skip_value = math.ceil(11000/new_base)
#             print("This is the skip value")
            
            for num in range(0,len(points)-skip_value,skip_value): 
                point = points[num]
                res = search.by_coordinate(float(point[0]),float(point[1]), radius=10, returns=1)
                if len(res) > 0:
                    codes.add(res[0].Zipcode)
#         print(codes)
        # quick empty check
        if len(codes) > 0:
            for x in codes:
                # need to reduce the number of API calls
                search_results_list.append(yelp_api.search_query(term=searchTerm,location=x,price="1,2",limit=10))
        restCount = 0
        # looks through all the search query results
        for search_results in search_results_list:
            for buis in search_results['businesses']:
                if searchTerm in buis['name']:
    #                 print(buis['name'])
                    if buis['id'] not in uniques:
                        uniques.add(buis['id'])
                        s = Shop(buis['coordinates']['latitude'],buis['coordinates']['longitude'],buis['id'])
                        shops.append(s)

        for shop_object in shops:
            for shop_object1 in shops:
                if shop_object.shop_id != shop_object1.shop_id and shop_object.visited == False and shop_object1.visited == False:
                    dist = calculate_distance(shop_object.lat,shop_object.lng,shop_object1.lat,shop_object1.lng)
                    if dist < 10:
#                         print("done")
                        shop_object.visited = True
        for x in shops:
            if x.visited == False:
                total = total + 1
                restCount = restCount + 1
        shops = list()
        codes = set()
        search_results_list = list()
        step.foodCount = restCount
        restCount = 0
    print('Route Distance')
    print((route.distance*3.28084)/5280)
    print('Route Duration')
    print(route.duration/3600)
    
    print('Total Restaraunts')
    print(total)
    print(len(uniques))
    
# def gather_sum_route(route):
#     sum = 0
#     for step in route.steps_list:
#         sum = sum + step.foodCount
#     route.route_score = sum

for rou in routes:
    print('NEW ROUTTTTTE')
    getCountsForRoute(rou)

# for route in routes:
#     gather_sum_route(route)

# for route in routes:
#     print(route.route_score)
    
        
        
        









# for route in b:
#     print('new rute')
#     for step in route.steps_list:
#         print(step.html)


NEW ROUTTTTTE
Route Distance
215.10069387121212
Route Duration
3.658333333333333
Total Restaraunts
33
65
NEW ROUTTTTTE
Route Distance
216.15764630303028
Route Duration
3.6630555555555557
Total Restaraunts
29
64
NEW ROUTTTTTE
Route Distance
225.6521984242424
Route Duration
3.7911111111111113
Total Restaraunts
27
62


In [9]:
import forecastio

darkSky_api_key = 'x'
lat = -31.967819
lng = 115.87718
forecast = forecastio.load_forecast(darkSky_api_key, lat, lng)

In [15]:
a = forecast.hourly()
for hourlyData in a.data:
        print(hourlyData.temperature)

14.31
14.71
15.46
16.48
16.87
16.46
16.43
16.33
16.02
15.68
15.41
15.17
15.14
15.05
14.97
14.92
14.52
13.87
13.38
12.97
12.93
12.99
12.42
11.53
10.97
10.91
11.29
11.85
12.14
12.5
13.04
13.52
13.57
13.27
12.73
12.27
11.99
11.94
11.91
11.74
11.28
10.86
10.63
10.54
10.46
10.17
9.57
9.07
8.92


In [35]:
z = polyline.decode('a}tmFjbf`QCc@?C?G?U?]?Y?a@ASAWCg@AQAMAMAGCMAMCOCOOm@Ma@IQEMKQKQKQCICIIMIKACmAyAyAgB[c@_@g@u@gAU_@[k@o@mAw@uAYi@S]QYW][a@YYc@]WSWM]Q[MYG?AYIYIWGg@I[E[Cc@?e@?}A@aABs@?w@?g@A}@Au@CCAC?A?_CGm@Ca@?aCCg@Ai@Ac@?S?k@A{BC]?q@?U@Q?g@?UAe@AQAY?IAM?UAk@Aq@Ag@?Q?Y@c@BYB_@BOBQ@UDUDWFc@Jq@PWFYHg@LKBKDmFjBm@RwAh@c@PcDnAk@T_@PYLq@VODSHy@^i@T{@Zu@VeBj@aB`@qAT_AJ]B}AHsC@{B?wBAaDAmAAkAA{CC{CCo@?qFGqCEeAKw@K}@Me@Kw@UICA?m@QUIc@Oe@Qc@SWMe@Y{@g@WQo@g@c@]wAqAiC}BuBmB}@y@yCiCkBcBuAmAi@c@i@i@[[GIEGKQKKk@g@KKCCAAY]A?UUAAEEKIECm@a@sAiAcBmAu@c@y@]}A_@[GGAcBQo@AqAA[?iCAaFCqC?uD?UAmD?mFAgDAoCAyHC_FAiFAoA?aBAuD?e@Am@AU?S?WAU?e@AWAWAUAUC]Ci@Cc@EaAIgAOy@Mi@Im@KSEi@KQEUI}@SoBa@YKc@Ko@OqAWg@ISC')

In [38]:
# for x in z:
#     z[x] = 'hi'
#     print(z[x])
    
# for coord in z:
#     print(coord[0])
#     print(coord[1])
print(len(z))

246


In [41]:
search = ZipcodeSearchEngine()
res = search.by_coordinate(38.867614, -94.705783, radius=10, returns=1)

In [45]:
res[0].Zipcode

'66221'